<h1 align='center'>Aplicando o aprendizado de máquina à análise de sentimentos</h1>

<p align='center'><img src= https://cibersistemas.pt/wp-content/uploads/2020/09/wall-5.jpeg width='500'></p>


Na era moderna da internet e das mídias sociais, as opiniões, avaliações e recomendações das pessoas se tornaram um recurso valioso para a ciência política e os negócios. Graças às tecnologias modernas, agora podemos coletar e analisar esses dados com mais eficiência. Neste capítulo, vamos nos aprofundar em um subcampo do processamento de linguagem natural (NLP) chamado análise de sentimentos e aprender a usar algoritmos de aprendizado de máquina para classificar documentos com base em sua polaridade: a atitude do escritor. Em particular, vamos trabalhar com um conjunto de dados de 50.000 críticas de filmes do Internet Movie Database (IMDb) e construir um preditor que possa distinguir entre críticas positivas e negativas.

## Preparando os dados de revisão de filme do IMDb para processamento de texto
Como mencionado, a análise de sentimentos, às vezes também chamada de **mineração de opinião**, é uma subdisciplina popular do campo mais amplo da *NLP*; preocupa-se em analisar a polaridade dos documentos. Uma tarefa popular na análise de sentimentos é a classificação de documentos com base nas opiniões ou emoções expressas dos autores em relação a um tópico específico.

In [1]:
import os
import sys
import tarfile
import time
import urllib.request


source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

# Extraindo os dados
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

100% | 80 MB | 1.00 MB/s | 80 sec elapsed

### Pré-processamento do conjunto de dados do filme em um formato mais conveniente
Tendo extraído com sucesso o conjunto de dados, agora reuniremos os documentos de texto individuais do arquivo de download descompactado em um único arquivo CSV.

A seguir, leremos as resenhas de filmes em um objeto *DataFrame* do *pandas*, que pode levar até 10 minutos em um computador desktop padrão.
Para visualizar o progresso e o tempo estimado até a conclusão, usaremos o pacote *Python Progress Indicator* (PyPrind, https://pypi.python.org/pypi/PyPrind/)que foi desenvolvido há vários anos para esses fins.

In [2]:
import pyprind
import pandas as pd
import os

# change the `basepath` to the directory of the
# unzipped movie dataset

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:49


In [4]:
# Para nossa própria conveniência, também armazenaremos o 
# conjunto de dados de revisão de filme montado e embaralhado como um arquivo CSV:

import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

df.to_csv('movie_data.csv', index=False, encoding='utf-8')